In [186]:
#Base libraries
import pandas as pd
import os
import re
import random
import numpy as np
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
import math as mt
import itertools

In [187]:
from osgeo import gdal
from descartes import PolygonPatch
from os.path import join
from shutil import copyfile

In [188]:
import numpy as np
import fiona
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
import rasterio.features
from shapely.geometry import shape, mapping
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import Polygon,Point

In [189]:
# geometries
import descartes
import shapely as shp
import rasterio
from rasterio.plot import show
from rasterio.features import shapes
from shapely.affinity import affine_transform
from shapely.geometry import Point, Polygon
from shapely.ops import triangulate

In [190]:
import geopandas as geo

In [191]:
import glob
import subprocess
from osgeo import gdal
import cv2
from ipyleaflet import Map, Marker, GeoJSON, GeoData

In [192]:
# This is required for me to have autocomplete
%config Completer.use_jedi = False
%matplotlib inline

In [193]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [194]:
# levers
make_wegen = False
make_1set = False
make_bounds = False

In [195]:
# Functions
def random_points_in_polygon(polygon, k):
    "Return list of k points chosen uniformly at random inside the polygon."
    areas = []
    transforms = []
    for t in triangulate(polygon):
        areas.append(t.area)
        (x0, y0), (x1, y1), (x2, y2), _ = t.exterior.coords
        transforms.append([x1 - x0, x2 - x0, y2 - y0, y1 - y0, x0, y0])
    points = []
    for transform in random.choices(transforms, weights=areas, k=k):
        x, y = [random.random() for _ in range(2)]
        if x + y > 1:
            p = Point(1 - x, 1 - y)
        else:
            p = Point(x, y)
        points.append(affine_transform(p, transform))
    return points

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]


# Functions for 1-set and 0-set pipeline
def find_overlap_tif(path_list, geometry):
    # Find general tifs that intersect 
    intersect_tifs = []
    for i in path_list:
        try:
            raster = rasterio.open(i)
            rasterio.features.geometry_window(raster, geometry)
        except:
            continue
        else:
            print("overlap found! saving")
            intersect_tifs.append(i)
    return(intersect_tifs)

def load_rasterset(raster_paths):
    dataset = []
    for i in raster_paths:
        rast = rasterio.open(i)
        dataset.append(rast)
    return(dataset)
    
def Gtif_to_XY(dataset):
    GtifXY = []
    
    for i in dataset:
        obj = i
        poly = Polygon([Point(obj.bounds.left,obj.bounds.top),Point(obj.bounds.right,obj.bounds.top),Point(obj.bounds.right,obj.bounds.bottom),Point(obj.bounds.left,obj.bounds.bottom)])
        GtifXY.append(poly)
        
    tiff_gd = geo.GeoDataFrame(geo.GeoSeries(GtifXY))
    tiff_gd.columns=['geometry']
    tiff_gd.set_geometry('geometry',inplace=True)
    return(tiff_gd)



def tif_geo_intersec(tif, shapes, id_col):
    geo_id = shapes[[id_col,"geometry"]]
    
    union = geo.overlay(tif, shapes, how='intersection')
    union2 = union[["path", id_col]]
    
    union_merge = union2.merge(geo_id, on=id_col)
    union_merge = geo.GeoDataFrame(union_merge,geometry = "geometry" ,crs = shapes.crs)
    return(union_merge)

## Loading required data

In [196]:
zebra_bord= pd.read_csv("data/L2_NDW_599.csv")
verkeerslichten= pd.read_csv("data/verkeerslichten.csv")
rotterdam_bb = geo.read_file("data/rotterdam_new_bb.shp")
wegen = geo.read_file("data/wegen_buffer.shp")

In [197]:
wegen = wegen.to_crs(crs=28992)

In [198]:
if make_wegen == True:
    wegen = geo.read_file("data/Wegvlakken_Clip.shp")
    # Make wegen geodata
    wegen = wegen.to_crs(epsg=32634)
    wegen_sim = wegen.simplify(tolerance=1)
    wegen["geometry"] = wegen_sim
    wegen["geometry"] = wegen.geometry.buffer(5 ,cap_style = 1)
    wegen.to_file("data/wegen_buffer.shp")

In [199]:
if make_bounds == True:
    # load data
    rotterdam_bb = geo.read_file("data/Gemeente_BB.shp")
    
    # Make rotterdam BB geodata
    bound = rotterdam_bb.bounds
    xmin = bound.minx.max()
    xmax = bound.maxx.max()
    ymin = bound.miny.max()
    ymax = bound.maxy.max()
    rotterdam_poly = shp.geometry.box(xmin, ymin, xmax, ymax)
    rotterdam_bb["geometry"] = rotterdam_poly
    
    # pad with -2000 meter
    rotterdam_bound = rotterdam_bb.copy()
    rotterdam_bound = rotterdam_bound.to_crs(epsg=32634)
    rotterdam_bound["geometry"] = rotterdam_bound.geometry.buffer(-2000, cap_style = 1)
    rotterdam_bound = rotterdam_bound.to_crs(epsg=28992)

    rotterdam_bound.to_file("data/rotterdam_new_bb.shp")

In [200]:
# make zebrabord geodata 
zebra_bord = geo.GeoDataFrame(zebra_bord,
                             geometry=geo.points_from_xy(zebra_bord.longitude, zebra_bord.latitude),
                            crs=4326)

In [201]:
zebra_bord = zebra_bord.to_crs(epsg=32634)

In [202]:
# make verkeerslichten geodata
verkeerslicht = geo.GeoDataFrame(verkeerslichten,
                            geometry=geo.points_from_xy(verkeerslichten.x, verkeerslichten.y),
                            crs=28992)

verkeerslicht = verkeerslicht.to_crs(epsg=32634)

## Create 1-set

### Get 1-set locations

In [31]:
if make_1set == True:
    # Maak 2 zebra-bord buffer sets
    zebra_buffer = zebra_bord.copy()
    zebra_buffer = zebra_buffer.to_crs(epsg=32634)
    zebra_buffer["geometry"] = zebra_buffer.geometry.buffer(25, cap_style = 3)
    zebra_buffer_2 = zebra_buffer.copy()
    
    # intersection df
    intersect_df = geo.overlay(zebra_buffer, zebra_buffer_2, how="intersection")
    intersect_df = intersect_df[["fid_1","fid_2"]]
    intersect_df2 = intersect_df.copy()
    intersect_l = intersect_df.fid_1.unique()
    
    # delete mirrored and double rows
    output = []
    for x in intersect_l:
        df_filt = intersect_df2[(intersect_df2.fid_1 == x)]

        remove_id = list(df_filt.fid_2)
        output.append(df_filt)

        intersect_df2 = intersect_df2[~intersect_df2['fid_1'].isin(remove_id)]
        intersect_df2 = intersect_df2[~intersect_df2['fid_2'].isin(remove_id)]
        
    # merge final grouping
    final_out = pd.concat(output)
    final_out.fid_1 = pd.factorize(final_out.fid_1)[0]
    final_out = final_out.rename(columns={"fid_2":"fid","fid_1":"group"})
    
    # merge and dissolve
    zebra_merged = zebra_bord.merge(final_out, on="fid")
    zebra_dissolved = zebra_merged.dissolve(by = "group")
    
    # set crs, get centroid of dissolved groups
    zebra_dissolved = zebra_dissolved.to_crs(epsg=32634)
    zebra_group_centroid = zebra_dissolved.copy()
    zebra_group_centroid.geometry = zebra_dissolved.geometry.centroid
    
    # create square buffer around each centroid that captures the image
    zebra_window = zebra_group_centroid.copy()
    zebra_window.geometry = zebra_group_centroid.geometry.buffer(25 ,cap_style = 3)
    zebra_window = zebra_window.to_crs(epsg=28992)
    
    # save 1-set locations
    zebra_window.to_file("data/1_set_locations.shp")

False

In [19]:
# # intersection df
# intersect_df = geo.overlay(zebra_buffer, zebra_buffer_2, how="intersection")
# intersect_df = intersect_df[["fid_1","fid_2"]]
# intersect_df2 = intersect_df.copy()
# intersect_l = intersect_df.fid_1.unique()

In [20]:
# # delete mirrored and double rows
# output = []
# for x in intersect_l:
#     df_filt = intersect_df2[(intersect_df2.fid_1 == x)]

#     remove_id = list(df_filt.fid_2)
#     output.append(df_filt)

#     intersect_df2 = intersect_df2[~intersect_df2['fid_1'].isin(remove_id)]
#     intersect_df2 = intersect_df2[~intersect_df2['fid_2'].isin(remove_id)]

In [21]:
# # merge final grouping
# final_out = pd.concat(output)
# final_out.fid_1 = pd.factorize(final_out.fid_1)[0]
# final_out = final_out.rename(columns={"fid_2":"fid","fid_1":"group"})

In [22]:
# # merge and dissolve
# zebra_merged = zebra_bord.merge(final_out, on="fid")
# zebra_dissolved = zebra_merged.dissolve(by = "group")

In [23]:
# # set crs, get centroid of dissolved groups
# zebra_dissolved = zebra_dissolved.to_crs(epsg=32634)
# zebra_group_centroid = zebra_dissolved.copy()
# zebra_group_centroid.geometry = zebra_dissolved.geometry.centroid

In [24]:
# # create square buffer around each centroid that captures the image
# zebra_window = zebra_group_centroid.copy()
# zebra_window.geometry = zebra_group_centroid.geometry.buffer(25 ,cap_style = 3)

In [25]:
# zebra_window = zebra_window.to_crs(epsg=28992)

In [27]:
# # save 1-set locations
# zebra_window.to_file("data/1_set_locations.shp")

C:\Users\sjoer\.conda\envs\raster\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


### check locations with geotiffs

In [13]:
# load data
zebra_window = geo.read_file("data/1_set_locations.shp")

In [10]:
# create geotiff index
tiff_folder = "data/GeoTIFF/"
mosaic_folder = "data/1_set_tiffs/"
tif_paths =[join(tiff_folder,f) for f in os.listdir(tiff_folder) if f.endswith('.tif')]

In [37]:
# Check for general overlapping tifs/geoms
intersect_tif = find_overlap_tif(tif_paths, zebra_window.geometry)

# load overlapping tifs
tif_data = load_rasterset(intersect_tif)

overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap fo

overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap fo

In [38]:
# convert tif to geopandas geodataframe
tif_gd = Gtif_to_XY(tif_data)

# add paths column
tif_gd["path"] = intersect_tif

# set CRS
tif_gd.set_crs(epsg=28992,inplace=True)

,geometry,path
0,"POLYGON ((61250.025 437250.000, 62000.025 4372...",data/GeoTIFF/2020_061250_436500_RGB_hr.tif
1,"POLYGON ((61250.025 438000.000, 62000.025 4380...",data/GeoTIFF/2020_061250_437250_RGB_hr.tif
2,"POLYGON ((61250.025 438750.000, 62000.025 4387...",data/GeoTIFF/2020_061250_438000_RGB_hr.tif
3,"POLYGON ((61250.025 439500.000, 62000.025 4395...",data/GeoTIFF/2020_061250_438750_RGB_hr.tif
4,"POLYGON ((61250.025 440250.000, 62000.025 4402...",data/GeoTIFF/2020_061250_439500_RGB_hr.tif
...,...,...
715,"POLYGON ((100250.025 442500.000, 101000.025 44...",data/GeoTIFF/2020_100250_441750_RGB_hr.tif
716,"POLYGON ((100250.025 443250.000, 101000.025 44...",data/GeoTIFF/2020_100250_442500_RGB_hr.tif
717,"POLYGON ((100250.025 444000.000, 101000.025 44...",data/GeoTIFF/2020_100250_443250_RGB_hr.tif
718,"POLYGON ((100250.025 444750.000, 101000.025 44...",data/GeoTIFF/2020_100250_444000_RGB_hr.tif


In [39]:
# tif and geo union
# union is the base file we use for further indexing
union = tif_geo_intersec(tif_gd, zebra_window, "fid")

In [40]:
# create mosaic where needed
fid_list = union.fid.unique()

In [43]:
for fid in fid_list:
    
    print(fid)
    
    #create filestring
    out_path = mosaic_folder+str(fid)+".tif"
    
    # get required polygon
    poly = union.loc[union.fid == fid]
    
    ## Fase 1: create mosaic if needed, else copy file
    # check for multiple rasters
    raster_paths = list(union["path"].loc[union.fid == fid])
    if len(raster_paths) > 1:
      
        # get required files for mosaic
        mosaic_data = []
        for i in raster_paths:
            src = rasterio.open(i)
            mosaic_data.append(src)
        
        # create mosaic and get old metadata
        mosaic, out_trans = merge(mosaic_data)
        out_meta = src.meta.copy()
        
        # update new metadta mosaic
        out_meta.update({"driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_trans})
        
        # write file
        with rasterio.open(out_path, "w", **out_meta) as dest:
            dest.write(mosaic)
            
    else:
        # just copy the file with new name
        copyfile(raster_paths[0], out_path)
    
    
    # Fase 2: Re-load new raster and clip with polygon
    # get required polygon
    poly = poly.iloc[[0],:]
    poly_feat = getFeatures(poly)
    
    # open raster data
    src = rasterio.open(out_path)
    out_meta = src.meta
    
    out_img, out_transform = mask(src, poly_feat, crop=True, filled=False)
    
    out_meta.update({"driver": "GTiff",
        "height": out_img.shape[1],
        "width": out_img.shape[2],
        "transform": out_transform})
    
    # write file
    with rasterio.open(out_path, "w", **out_meta) as dest:
        dest.write(out_img)

185
293
340
3730
47422
345
339
21907
21973
20913
20919
21023
21029
21031
21033
21964
337
21025
20915
20917
21028
21034
21037
21399
21810
21968
20911
21024
52792
52793
52795
21030
21042
21132
21133
21459
21464
21818
21972
21976
21975
22001
44273
43496
43548
43498
44228
43536
43542
43529
43539
43552
43556
43557
43774
43528
43534
4352
43492
43494
43533
43535
43540
344
2533
23542
22217
22309
22311
22315
22317
22401
23901
22419
22425
22400
22325
22399
22411
22423
22396
22421
22440
23257
22214
22398
22406
23921
24093
22412
22418
22424
22446
24020
22323
22410
22252
22254
22321
22327
22394
23255
24372
53053
22427
22428
23118
24012
22407
22216
22395
22416
37369
55850
37368
37371
11908
44492
44497
44496
44494
10853
10324
32270
11925
11947
13572
13580
13600
44495
35191
35199
35211
35213
35221
35238
35239
35052
35187
35192
36778
35194
35206
35210
45826
11853
11934
11937
12915
11858
11862
11873
11875
11879
11881
11884
11887
11898
11902
11932
13807
32575
44493
35207
35208
35212
36367
35241
35517
351

## Generate 0-set locations

In [203]:
# load data
zebra_window = geo.read_file("data/1_set_locations.shp")
rotterdam_bb = geo.read_file("data/rotterdam_new_bb.shp")

In [204]:
bound = rotterdam_bb.bounds
xmin = bound.minx.max()
xmax = bound.maxx.max()
ymin = bound.miny.max()
ymax = bound.maxy.max()

In [205]:
# generate random point
x = []
y = []
for i in range(20000):
    x.append(random.uniform(xmin, xmax))
    y.append(random.uniform(ymin, ymax))

df = pd.DataFrame({"x":x,
                  "y":y})
df = df.drop_duplicates()
point = geo.GeoDataFrame(df, geometry=geo.points_from_xy(df.x, df.y), crs = 28992)

In [206]:
# create buffer window from point
point = point.to_crs(epsg=32634)
point["geometry"] = point.geometry.buffer(25, cap_style = 3)
point = point.to_crs(epsg=28992)

In [207]:
point["id"] = range(point.shape[0])
point = point.drop(["x","y"], axis=1)

In [208]:
# Step 0: Check if inside bounds of Rotterdam
overlap_0 = geo.overlay(rotterdam_bb, point, how='intersection')

In [209]:
# get overlapping id's
overlap_id = overlap_0.id.unique()

# subselect overlapping id's
point_in = point[point.id.isin(overlap_id)]

In [210]:
# Step 1:Get windows that don't overlap 1-set
overlap_1 = geo.overlay(zebra_window[["fid","geometry"]], point_in, how='intersection')

In [211]:
# get overlapping id's
overlap_id = overlap_1.id.unique()

# subselect non_overlapping id's
point_sub = point[~point.id.isin(overlap_id)]

In [212]:
# Step 2: Get windows that overlap roads

In [213]:
overlap_roads = geo.overlay(wegen[["WVK_ID","geometry"]], point_sub, how='intersection')

In [214]:
# get overlapping id's
overlap_id = overlap_roads.id.unique()

In [215]:
# split set in roads and non-roads
road_window = point_sub[point_sub.id.isin(overlap_id)]
other_window = point_sub[~point_sub.id.isin(overlap_id)]

In [216]:
# pick 800 road, 400 non-road
r1 = road_window.sample(n=4000)
r2 = other_window.sample(n=600)

In [217]:
null_set = geo.GeoDataFrame( pd.concat([r1,r2], ignore_index=True), crs = r1.crs)

In [218]:
null_set.head()

,geometry,id
0,"POLYGON ((86999.531 444791.675, 87010.138 4447...",4680
1,"POLYGON ((83204.077 438053.228, 83214.674 4380...",19928
2,"POLYGON ((83187.019 436326.449, 83197.614 4362...",1334
3,"POLYGON ((69214.462 437714.357, 69225.059 4376...",9975
4,"POLYGON ((74705.563 444863.712, 74716.170 4448...",13115


In [219]:
null_set.shape

(4600, 2)

In [220]:
# create geotiff index
tiff_folder = "data/GeoTIFF/"
mosaic_folder = "data/0_set_tiffs/"
tif_paths =[join(tiff_folder,f) for f in os.listdir(tiff_folder) if f.endswith('.tif')]

In [221]:
# Check for general overlapping tifs/geoms
intersect_tif = find_overlap_tif(tif_paths, null_set.geometry)

# load overlapping tifs
tif_data = load_rasterset(intersect_tif)

overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap fo

overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap fo

overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving
overlap found! saving


In [222]:
# convert tif to geopandas geodataframe
tif_gd = Gtif_to_XY(tif_data)

# add paths column
tif_gd["path"] = intersect_tif

# set CRS
tif_gd.set_crs(epsg=28992,inplace=True)

,geometry,path
0,"POLYGON ((56750.025 438750.000, 57500.025 4387...",data/GeoTIFF/2020_056750_438000_RGB_hr.tif
1,"POLYGON ((56750.025 439500.000, 57500.025 4395...",data/GeoTIFF/2020_056750_438750_RGB_hr.tif
2,"POLYGON ((56750.025 440250.000, 57500.025 4402...",data/GeoTIFF/2020_056750_439500_RGB_hr.tif
3,"POLYGON ((56750.025 441000.000, 57500.025 4410...",data/GeoTIFF/2020_056750_440250_RGB_hr.tif
4,"POLYGON ((56750.025 441750.000, 57500.025 4417...",data/GeoTIFF/2020_056750_441000_RGB_hr.tif
...,...,...
783,"POLYGON ((98750.025 442500.000, 99500.025 4425...",data/GeoTIFF/2020_098750_441750_RGB_hr.tif
784,"POLYGON ((98750.025 443250.000, 99500.025 4432...",data/GeoTIFF/2020_098750_442500_RGB_hr.tif
785,"POLYGON ((98750.025 444000.000, 99500.025 4440...",data/GeoTIFF/2020_098750_443250_RGB_hr.tif
786,"POLYGON ((98750.025 444750.000, 99500.025 4447...",data/GeoTIFF/2020_098750_444000_RGB_hr.tif


In [223]:
# tif and geo union
# union is the base file we use for further indexing
union = tif_geo_intersec(tif_gd, null_set, "id")

In [224]:
# create mosaic where needed
id_list = union.id.unique()

In [225]:
len(id_list)

3016

In [226]:
dim=783
remove_list = []
for fid in id_list:
    
    print(fid)
    
    #create filestring
    out_path = mosaic_folder+str(fid)+".tif"
    
    # get required polygon
    poly = union.loc[union.id == fid]
    
    ## Fase 1: create mosaic if needed, else copy file
    # check for multiple rasters
    raster_paths = list(union["path"].loc[union.id == fid])
    if len(raster_paths) > 1:
      
        # get required files for mosaic
        mosaic_data = []
        for i in raster_paths:
            src = rasterio.open(i)
            mosaic_data.append(src)
        
        # create mosaic and get old metadata
        mosaic, out_trans = merge(mosaic_data)
        out_meta = src.meta.copy()
        
        
        # update new metdata mosaic
        out_meta.update({"driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_trans})
        
        # write file
        with rasterio.open(out_path, "w", **out_meta) as dest:
            dest.write(mosaic)
            
    else:
        # just copy the file with new name
        copyfile(raster_paths[0], out_path)
    
    
    # Fase 2: Re-load new raster and clip with polygon
    # get required polygon
    poly = poly.iloc[[0],:]
    poly_feat = getFeatures(poly)
    
    # open raster data
    src = rasterio.open(out_path)
    out_meta = src.meta.copy()
    
    out_img, out_transform = mask(src, poly_feat, crop=True, filled=False)

    if(out_img.shape[1] == dim)&(out_img.shape[2] == dim):
        out_meta.update({"driver": "GTiff",
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform})

        # write file
        with rasterio.open(out_path, "w", **out_meta) as dest:
            dest.write(out_img)
    else:
        src.close()
        os.remove(out_path)

10181
13630
2914
3521
18515
3845
3158
3778
1337
19284
6800
15778
8186
1499
13761
16150
3783
11929
602
11252
4561
16572
2791
19506
5223
10594
16039
1574
7027
3724
12632
15418
17877
16946
293
13744
11025
2569
15868
7106
9478
8641
15134
19135
10906
17995
4259
3276
4061
3586
13683
11077
18428
12209
764
15429
4327
13282
8730
19136
5623
17946
364
16594
7357
9143
9774
3452
2091
14242
18245
11009
2480
14196
15714
4798
14277
9373
11297
11001
4487
8100
2048
19388
10826
9932
6869
6123
10229
7631
3646
1565
15396
2402
9711
10507
18987
5504
3538
18094
3013
9516
4121
9338
17849
17708
18589
11749
16412
15653
3972
16062
5272
16320
17570
7022
4421
12285
7241
1127
13846
10418
9717
8397
6594
4100
15178
576
19776
11530
2221
4640
4167
15951
19915
716
18218
5374
13068
14507
1441
17217
17364
6898
8410
4445
1142
18751
10743
9181
16710
8198
1104
5460
18085
18501
8851
9973
3993
7149
8715
19456
9176
3160
7050
15816
6316
11022
503
4452
11234
4980
14589
14352
19527
19757
19808
9158
316
15956
3313
17737
13313
1791
1

KeyboardInterrupt: 

Exception ignored in: 'rasterio._env.log_error'
Traceback (most recent call last):
  File "C:\Users\sjoer\.conda\envs\raster\lib\logging\__init__.py", line 1934, in getLogger
    if name:
KeyboardInterrupt


18183
19555
6163


KeyboardInterrupt: 

## leaflet

In [76]:
zebra_bord_2 = zebra_bord.copy()
#zebra_bord_2 = zebra_bord_2.to_crs(epsg=32634)
#zebra_bord_2["geometry"] = zebra_bord_2.geometry.buffer(1, cap_style = 1)
zebra_bord_2 = zebra_bord_2.to_crs(epsg=4326)

In [26]:
zebra_buffer = zebra_window.to_crs(epsg=4326)

In [69]:
zebra_center = zebra_window.copy()
zebra_center = zebra_center.to_crs(epsg=32634)
zebra_center["geometry"] = zebra_window.geometry.centroid
#zebra_center["geometry"] = zebra_center.geometry.buffer(1, cap_style = 1)
zebra_center = zebra_center.to_crs(epsg=4326)

In [92]:
zebra_leaf = GeoData(geo_dataframe=zebra_bord_2,
                      style={'color': 'black', 'radius':8, 'fillColor': '#336eff', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                      hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                      point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'yellow', 'weight': 3})

In [127]:
zebra_buffer_leaf = GeoData(geo_dataframe=zebra_buffer,
                           style={"color":"#7bd85f", 
                                  "opacity": 2, "weight": 5, 'fillOpacity':0.4})

In [108]:
center_leaf = GeoData(geo_dataframe=zebra_center,
                      style={'color': 'black', 'radius':8, 'fillColor': '#ffbb33', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                      hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                      point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'yellow', 'weight': 3})

In [ ]:
zebra_leaf = GeoData(geo_dataframe=zebra_bord)
zebra_buffer_leaf = GeoData(geo_dataframe=zebra_buffer)
zebra_group_centroid_leaf = GeoData(geo_dataframe=zebra_group_centroid)

licht_leaf = GeoData(geo_dataframe=verkeerslicht)
wegen_leaf = GeoData(geo_dataframe=wegen)
bb_leaf = GeoData(geo_dataframe=rotterdam_bb)
point_leaf = GeoData(geo_dataframe=point)

In [149]:
rotterdam_leaf1 = rotterdam_bound
rotterdam_leaf1 = rotterdam_leaf1.to_crs(crs=4326)
rotterdam_leaf1 = GeoData(geo_dataframe=rotterdam_leaf1)

In [ ]:
#zebra_filter = zebra_bord[zebra_bord["fid"].isin([293,294,295,296,297,298,338,341])]
zbr_leaf = GeoData(geo_dataframe=zebra_filter)

In [ ]:
wegen_leaf = wegen.to_crs(crs=4326)
wegen_leaf = GeoData(geo_dataframe=wegen_leaf)

In [ ]:
zebra_leaf = zebra_window.to_crs(epsg=4326)
zebra_leaf = GeoData(geo_dataframe=zebra_leaf)

In [49]:
road_window_leaf = road_window.to_crs(crs=4326)
road_window_leaf = GeoData(geo_dataframe=road_window_leaf)

In [ ]:
union_leaf = union.loc[union.fid == 19361]
union_leaf = union_leaf.to_crs(epsg=4326)
union_leaf = GeoData(geo_dataframe=union_leaf)

In [150]:
center = (51.915039, 4.445772)
m = Map(center=center, zoom=25)

m.add_layer(rotterdam_leaf1)
#m.add_layer(center_leaf)
#m.add_layer(zebra_buffer_leaf)

In [151]:

m

Map(center=[51.915039, 4.445772], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [ ]:
# Get list of geotiffs
tiff_list = os.listdir("data/GeoTIFF/")

In [ ]:
tiff_list[1:20]

In [ ]:
# make NDW data 
ndw_shape = geo.GeoDataFrame(NDW,
                             geometry=geo.points_from_xy(NDW.longitude, NDW.latitude))

In [ ]:
roads = ndw_shape.dissolve(by = "schemaVersion", aggfunc='sum')